In [1]:
import os,sys,inspect
import json
import torch
currentdir = os.path.dirname(os.getcwd())
print(currentdir)
# parent_dir = os.path.abspath(os.path.join(currentdir, os.pardir))
root = os.path.dirname(currentdir)
print("root Folder Absoloute path: ", root)


import sys
sys.path.append(root)

import logging

logging.basicConfig(level=logging.INFO)

from IPython.display import Image

import __main__

__main__.__file__="main.py"

/home/hfaghihi/Framework/new/DomiKnowS/examples
root Folder Absoloute path:  /home/hfaghihi/Framework/new/DomiKnowS


In [2]:
from regr.data.reader import RegrReader
import torch


class SudokuReader(RegrReader):
    def parse_file(self):
        return [["random"] * 1]
    def getidval(self, item):
        return [1]
    def getwhole_sudokuval(self, item):
        base  = 3
        side  = base*base

        # pattern for a baseline valid solution
        def pattern(r,c): return (base*(r%base)+r//base+c)%side

        # randomize rows, columns and numbers (of valid base pattern)
        from random import sample
        def shuffle(s): return sample(s,len(s)) 
        rBase = range(base) 
        rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
        cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
        nums  = shuffle(range(1,base*base+1))

        # produce board using randomized baseline pattern
        board = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
        squares = side*side
        empties = squares * 3//4
        for p in sample(range(squares),empties):
            board[p//side][p%side] = 0
        return torch.tensor(board)
            
    
    def getsizeval(self, item):
        return 9, 9
    

In [3]:
trainreader = SudokuReader("randn", type="raw")
item = next(iter(trainreader))
print(item)
indices = (item['whole_sudoku'] != 0).nonzero()
print(indices)
print(torch.where(item['whole_sudoku'] != 0))
print(item['whole_sudoku'][indices[:, 0], indices[:, 1]] )

{'id': [1], 'size': (9, 9), 'whole_sudoku': tensor([[0, 0, 0, 3, 0, 0, 0, 0, 0],
        [0, 0, 8, 0, 4, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 6, 0, 0, 0],
        [6, 0, 3, 0, 5, 0, 0, 0, 0],
        [9, 0, 0, 0, 0, 0, 5, 0, 1],
        [0, 0, 0, 9, 0, 0, 0, 0, 0],
        [8, 5, 9, 0, 0, 0, 0, 3, 2],
        [2, 3, 1, 0, 0, 0, 0, 7, 0],
        [0, 0, 6, 0, 0, 0, 0, 0, 0]])}
tensor([[0, 3],
        [1, 2],
        [1, 4],
        [2, 5],
        [3, 0],
        [3, 2],
        [3, 4],
        [4, 0],
        [4, 6],
        [4, 8],
        [5, 3],
        [6, 0],
        [6, 1],
        [6, 2],
        [6, 7],
        [6, 8],
        [7, 0],
        [7, 1],
        [7, 2],
        [7, 7],
        [8, 2]])
(tensor([0, 1, 1, 2, 3, 3, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8]), tensor([3, 2, 4, 5, 0, 2, 4, 0, 6, 8, 3, 0, 1, 2, 7, 8, 0, 1, 2, 7, 2]))
tensor([3, 8, 4, 6, 6, 3, 5, 9, 5, 1, 9, 8, 5, 9, 3, 2, 2, 3, 1, 7, 6])


main.py:4: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729006826/work/torch/csrc/utils/python_arg_parser.cpp:882.)


In [4]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, V, exactL
from regr.graph import EnumConcept


Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    sudoku = Concept("sodoku")
    
    empty_entries = Concept(name="empty_entries")
    fixed_entries = Concept(name="fixed_entries")
    
    (sudoku_empty, sudoku_fixed) = sudoku.has_a(empty_entries, fixed_entries)
    
    empty_entry = Concept(name='empty_entry')
    (empty_rel, ) = empty_entries.contains(empty_entry)
    
    fixed_entry = Concept(name='fixed_entry')
    (fixed_rel, ) = fixed_entries.contains(fixed_entry)
    
    same_row = Concept(name="same_row")
    same_row_mixed = Concept(name="same_row_mixed")
    (same_row_arg1, same_row_arg2) = same_row.has_a(arg1=empty_entry, arg2=empty_entry)
    (same_row_mixed_arg1, same_row_mixed_arg2) = same_row_mixed.has_a(arg1=empty_entry, arg2=fixed_entry)
    
    same_col = Concept(name="same_col")
    same_col_mixed = Concept(name="same_col_mixed")
    (same_col_arg1, same_col_arg2) = same_col.has_a(arg1=empty_entry, arg2=empty_entry)
    (same_col_mixed_arg1, same_col_mixed_arg2) = same_col_mixed.has_a(arg1=empty_entry, arg2=fixed_entry)
    
    empty_entry_label = empty_entry(name="empty_entry_label", ConceptClass=EnumConcept, values=["v0", "v1", "v2", "v3", "v4", "v5", "v6", "v7", "v8", "v9", "v10"])


    ### Constriants
    
    for val in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]:
        ### No same number in the same row between empty entries and empty entries and the fixed ones.
        ifL(getattr(empty_entry_label, f'v{val}')('x'), notL(orL(
            existsL(
                andL(
                    same_row('z', path=("x", same_row_arg1.reversed)), getattr(empty_entry_label, f'v{val}')('y', path=("z", same_row_arg2))
                )), 
            existsL(
                andL(
                    same_row_mixed('z', path=("x", same_row_mixed_arg1.reversed)), getattr(empty_entry_label, f'v{val}')('y', path=("z", same_row_mixed_arg2))
                ))
        )))
        
        ### No same number in the same column between empty entries and empty entries and the fixed ones.
        ifL(getattr(empty_entry_label, f'v{val}')('x'), notL(orL(
            existsL(
                andL(
                    same_col('z', path=("x", same_col_arg1.reversed)), getattr(empty_entry_label, f'v{val}')('y', path=("z", same_col_arg2))
                )), 
            existsL(
                andL(
                    same_col_mixed('z', path=("x", same_col_mixed_arg1.reversed)), getattr(empty_entry_label, f'v{val}')('y', path=("z", same_col_mixed_arg2))
                ))
        )))


Log file for dataNode is in: /home/hfaghihi/Framework/new/DomiKnowS/examples/sudoku/logs/datanode.log


main.py:26: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'empty_entry'), ('arg2', 'empty_entry')]) is used.
main.py:27: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'empty_entry'), ('arg2', 'fixed_entry')]) is used.
main.py:31: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'empty_entry'), ('arg2', 'empty_entry')]) is used.
main.py:32: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'empty_entry'), ('arg2', 'fixed_entry')]) is used.


In [5]:
from regr.sensor.pytorch.sensors import FunctionalSensor, JointSensor, ReaderSensor, FunctionalReaderSensor
from regr.sensor.pytorch.learners import ModuleLearner
from regr.sensor.pytorch.relation_sensors import CompositionCandidateSensor
from regr.sensor.pytorch.query_sensor import DataNodeReaderSensor

class JointFunctionalReaderSensor(JointSensor, FunctionalReaderSensor):
    pass


def getempties(*prev, data):
    rows, cols = torch.where(data == 0)
    rel = torch.ones(1, 1)
    return [rows], [cols]
    
def getfixed(*prev, data):
    rows, cols = torch.where(data != 0)
    rel = torch.ones(1, 1)
    return [rows], [cols]
    
empty_entries['rows', 'cols'] = JointFunctionalReaderSensor(keyword='whole_sudoku', forward=getempties)
fixed_entries['rows1', 'cols'] = JointFunctionalReaderSensor(keyword='whole_sudoku', forward=getfixed)


def makesudoku(*prev, data):
    return [id], torch.ones(1, 1), torch.ones(1, 1)
sudoku['id', sudoku_empty.reversed, sudoku_fixed.reversed] = JointFunctionalReaderSensor(empty_entries['rows'], fixed_entries['rows1'], keyword='id', forward=makesudoku)


def make_empty_indexes(*prev):
    size = prev[0][0].shape[0]
    rel = torch.ones(size, 1)
    return rel, prev[0][0],prev[1][0]

empty_entry[empty_rel, 'row', 'column'] = JointSensor(empty_entries['rows'], empty_entries['cols'], forward=make_empty_indexes)

def read_fix_values(*prev, data):
    values = data[prev[0][0], prev[1][0]]
    size = prev[0][0].shape[0]
    rel = torch.ones(size, 1)
    return rel, prev[0][0], prev[1][0], values

fixed_entries[fixed_rel, 'row', 'column', 'value'] = JointFunctionalReaderSensor(fixed_entries['rows1'], fixed_entries['cols'], keyword='whole_sudoku', forward=read_fix_values)


### What kind of model should we use for learning the entries? Because it should be aware of all other decision to make the correct decision, otherwise it is impossible for the model to learn good weights.


